<a href="https://colab.research.google.com/github/ShubhamP1028/Crop-Disease-Detection/blob/main/CropDiseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/spMohanty/PlantVillage-Dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163235, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.


In [ ]:
DATASET_ROOT = "/content/PlantVillage-Dataset/raw/color"

In [ ]:
!ls $DATASET_ROOT | head

In [ ]:
# !pip install tensorflow tensorflow-datasets matplotlib seaborn scikit-learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="training",
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    DATASET_ROOT,
    target_size=img_size,
    batch_size=batch_size,
    subset="validation",
    class_mode="categorical"
)

In [ ]:
num_classes = len(train_gen.class_indices)
print("Classes:", train_gen.class_indices)

Build Model (Transfer Learning)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base initially


Building custom classifier head

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# Batch Normalisation
x = tf.keras.layers.BatchNormalization()(x)

In [ ]:
# Dropout for regularization
x = tf.keras.layers.Dropout(0.4)(x)

In [ ]:
x = tf.keras.layers.Dense(256, activation="relu")(x)
# Batch Normalisation after dense layer
x = tf.keras.layers.BatchNormalization()(x)

In [ ]:
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

In [ ]:
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

Compile Model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class GradientLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_gradients = []

    def on_train_batch_end(self, batch, logs=None):
        with tf.GradientTape() as tape:
            y_pred = self.model(self.model.train_function.inputs[0])
            loss = self.model.compiled_loss(
                self.model.train_function.inputs[1], y_pred
            )
        grads = tape.gradient(loss, self.model.trainable_weights)
        # Collect average gradient magnitude per batch
        grad_norm = np.mean([tf.reduce_mean(tf.abs(g)).numpy() for g in grads if g is not None])
        self.batch_gradients.append(grad_norm)

    def on_epoch_end(self, epoch, logs=None):
        plt.plot(self.batch_gradients, label=f"Epoch {epoch+1}")
        plt.title("Gradient Magnitudes per Batch")
        plt.xlabel("Batch")
        plt.ylabel("Average |Gradient|")
        plt.legend()
        plt.show()
        self.batch_gradients = []  # Reset for next epoch
